# 基本问题

In [1]:
import numpy as np

In [5]:
a = np.array([[1,2],[3,4]])
print(a)

[[1 2]
 [3 4]]


In [9]:
sum0 = np.sum(a,axis=0) # axis = 0 表示对每列求和
sum1 = np.sum(a,axis=1) # axis = 1 表示对每行求和
sum = np.sum(a) # 对所有元素求和，默认axis=None

In [10]:
print(sum0)
print(sum1)
print(sum)

[4 6]
[3 7]
10


## 彩色图片通道问题：
    1.tensorflow通道在后,(100,16,32,3)表示100张RGB三通道的16*32彩色图片
    2.theano/caffe则采用通道在前，(100,3,16,32)

## batch 
    Keras中的优化器SDG是随机梯度下降法的缩写,但不代表是一个样本就更新一回，还是基于mini-batch的。

## Keras调用GPU
    采用tensorflow作为后端，当机器有可用的GPU时，代码会自动调用GPU进行并行运算

## 保存Keras模型
    使用model.save(filepath)将权重保存在一个HDF5文件中，该文件包含：
    ·模型的结构，以便重构该模型
    ·模型的权重
    ·训练配置（损失函数，优化器等）
    ·优化器的状态，以便从上次训练中断的地方开始

### 保存模型的全部
    keras.models import load_model
    model.save('my_model.h5')
    del model # deletes the existing model
    model = load_model('my_model.h5') # return a compiled model

### 只希望保存模型的结构，而不包含其权重或配置信息
    json_string = model.to_json()# save as JSON
    yaml_string = model.to_yaml()# save as yaml
    
    # 从保存好的json文件或yaml文件中载入文件
    from keras.models import model_from_json,model_from_yaml
    model = model_from_json(json_string)
    model = model_from_yaml(yaml_string)

### 保存模型的权重
    model.save_weights('my_model_weights.h5')
    # 在代码中初始化一个完全相同的模型
    model.load_weights('my_mode_weights.h5')
    # 需要加载权重到不同的网络结构中（有些层一样）中，例如fine-tune，可以通过层的名字来加载模型：
    model.load_weights('my_model_weights.h5',by_name = True)
    
    """
    假如原模型为：
    model = Sequential()
    model.add(Dense(2, input_dim=3, name="dense_1"))
    model.add(Dense(3, name="dense_2"))
    ...
    model.save_weights(fname)
    """
    # new model
    model = Sequential()
    model.add(Dense(2, input_dim=3, name="dense_1"))  # will be loaded
    model.add(Dense(10, name="new_dense"))  # will not be loaded

    # load weights from first model; will only affect the first layer, dense_1.
    model.load_weights(fname, by_name=True)

## 获取中间层的输出
    from keras.models import Model
    model = ...# create the original model
    layer_name = 'my_layer'
    intermediate_layer_model = Model(input=model.input,output=model.get_layer(layer_name).output)
    intermdediate_output = intermediate_layer_model.predict(data)

## 利用keras处理超过机器内存的数据集
- model.train_on_batch(X,y)和model.test_on_batch(X,yield) 

- 也可以编写一个每次产生一个batch样本的生成器函数，并调用
  model.fit_generator(data_generator, samples_per_epoch, nb_epoch)

## 当验证集loss不在下降时，中断训练
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=2)
    model.fit(X, y, validation_split=0.2, callbacks=[early_stopping])

## 验证集如何从训练集中分割出来
    在model.fit中设置validation_spilt的值

## 训练数据在训练时会被随机洗乱吗
    设置model.fit的参数shuffle为真，训练数据就会被随机洗乱。不设置时默认为真。训练数据会在每个epoch的训练都重新洗乱一次。

## 如何在每个epoch后记录训练/测试的loss的正确率
    model.fit运行结束后返回一个History对象，其中history属性包含了训练过程中的损失函数的值以及其他度量指标。
    hist = model.fit(X,y,validation_split=0.2)
    print(hist.history)

## 从Sequential模型中除去一个层
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=784))
    model.add(Dense(32, activation='relu'))

    print(len(model.layers))  # "2"

    model.pop()
    print(len(model.layers))  # "1"

## 如何自keras中使用预训练的模型
提供下列图像分类的模型代码及预训练权重
- VGG16
- VGG19
- ResNet50
- inception v3

    from keras.applications.vgg16 import VGG16
    from keras.applications.vgg19 import VGG19
    from keras.applications.resnet50 import ResNet50
    from keras.applications.inception_v3 import InceptionV3

    model = VGG16(weights='imagenet', include_top=True)